# 🔍 Apache Spark vs DuckDB: Performance Comparison

This notebook benchmarks SQL query execution performance between **Apache Spark** and **DuckDB** on the same datasets in a single-node environment. The objective is to test runtime, usability, and functionality of both engines using real-world JSON data.

**Dataset 1**: ADS-B telemetry data from aircraft  
**Dataset 2**: Aircraft metadata keyed by `hex` ID

We will:
- Load both datasets into Spark and DuckDB
- Run identical SQL queries
- Compare performance and ease of use


In [1]:
# Core libraries
import pandas as pd
import duckdb
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Confirm environment
import platform
print("Python version:", platform.python_version())


Python version: 3.12.1


In [2]:
# File paths
small_telemetry_path = "D:/Project/Spark_project/combined_file_small.txt"
large_telemetry_path = "D:/Project/Spark_project/combined_file_large.txt"
aircraft_info_path = "D:/GitHub/Apache_spark_/spark/basic-ac-db.json"

one_day_path = "D:/Project/Spark_project/one_day_data.txt"

In [3]:
# 🚀 Initialize Spark Session
spark = SparkSession.builder \
    .appName("SparkDuckDB_Comparison") \
    .master("local[*]") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

print("✅ Spark Session created")

# 🐥 Initialize DuckDB In-Memory Connection
con = duckdb.connect(database=':memory:')
print("✅ DuckDB In-Memory Session initialized")


✅ Spark Session created
✅ DuckDB In-Memory Session initialized


In [4]:
# Load full telemetry and extract all payload fields in Spark
df_spark_small = spark.read.json(small_telemetry_path)

df_spark_small_flat = df_spark_small.select(
    col("dt").alias("timestamp"),
    col("payload.hex").alias("hex"),
    col("payload.alt_baro").alias("alt_baro"),
    col("payload.alt_geom").alias("alt_geom"),
    col("payload.gs").alias("ground_speed"),
    col("payload.track").alias("track"),
    col("payload.baro_rate").alias("baro_rate"),
    col("payload.squawk").alias("squawk"),
    col("payload.emergency").alias("emergency"),
    col("payload.category").alias("category"),
    col("payload.nav_qnh").alias("nav_qnh"),
    col("payload.nav_altitude_mcp").alias("nav_altitude_mcp"),
    col("payload.nav_heading").alias("nav_heading"),
    col("payload.lat").alias("lat"),
    col("payload.lon").alias("lon"),
    col("payload.nic").alias("nic"),
    col("payload.rc").alias("rc"),
    col("payload.seen_pos").alias("seen_pos"),
    col("payload.version").alias("version"),
    col("payload.nic_baro").alias("nic_baro"),
    col("payload.nac_p").alias("nac_p"),
    col("payload.nac_v").alias("nac_v"),
    col("payload.sil").alias("sil"),
    col("payload.sil_type").alias("sil_type"),
    col("payload.gva").alias("gva"),
    col("payload.sda").alias("sda"),
    col("payload.mlat").alias("mlat"),
    col("payload.tisb").alias("tisb"),
    col("payload.messages").alias("messages"),
    col("payload.seen").alias("seen"),
    col("payload.rssi").alias("rssi"),
    col("payload.flight").alias("flight")
)

df_spark_small_flat.show(5)


+--------------------+------+--------+--------+------------+-----+---------+------+---------+--------+-------+----------------+-----------+---------+----------+---+---+--------+-------+--------+-----+-----+---+--------+---+---+----+----+--------+----+-----+--------+
|           timestamp|   hex|alt_baro|alt_geom|ground_speed|track|baro_rate|squawk|emergency|category|nav_qnh|nav_altitude_mcp|nav_heading|      lat|       lon|nic| rc|seen_pos|version|nic_baro|nac_p|nac_v|sil|sil_type|gva|sda|mlat|tisb|messages|seen| rssi|  flight|
+--------------------+------+--------+--------+------------+-----+---------+------+---------+--------+-------+----------------+-----------+---------+----------+---+---+--------+-------+--------+-----+-----+---+--------+---+---+----+----+--------+----+-----+--------+
|2025-01-31 00:00:...|ab35d3|   37000|   36475|       552.7| 41.3|        0|  5740|     none|      A5| 1013.6|           36992|        0.0|44.218048|-75.741316|  8|186|     0.4|      2|       1|    9

In [ ]:
# from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, BooleanType

# payload_schema = StructType([
#     StructField("hex", StringType()),
#     StructField("alt_baro", DoubleType()),
#     StructField("alt_geom", DoubleType()),
#     StructField("gs", DoubleType()),
#     StructField("track", DoubleType()),
#     StructField("baro_rate", DoubleType()),
#     StructField("squawk", StringType()),
#     StructField("emergency", StringType()),
#     StructField("category", StringType()),
#     StructField("nav_qnh", DoubleType()),
#     StructField("nav_altitude_mcp", DoubleType()),
#     StructField("nav_heading", DoubleType()),
#     StructField("lat", DoubleType()),
#     StructField("lon", DoubleType()),
#     StructField("nic", IntegerType()),
#     StructField("rc", IntegerType()),
#     StructField("seen_pos", DoubleType()),
#     StructField("version", IntegerType()),
#     StructField("nic_baro", IntegerType()),
#     StructField("nac_p", IntegerType()),
#     StructField("nac_v", IntegerType()),
#     StructField("sil", IntegerType()),
#     StructField("sil_type", StringType()),
#     StructField("gva", IntegerType()),
#     StructField("sda", IntegerType()),
#     StructField("mlat", BooleanType()),
#     StructField("tisb", BooleanType()),
#     StructField("messages", IntegerType()),
#     # StructField("seen", DoubleType()),
#     StructField("rssi", DoubleType()),
#     StructField("flight", StringType())
# ])


In [ ]:
# from pyspark.sql.functions import from_json

# df_spark_large_raw = spark.read.json(large_telemetry_path)

# df_spark_large = df_spark_large_raw.withColumn(
#     "parsed_payload", from_json("payload", payload_schema)
# ).select(
#     col("dt").alias("timestamp"),
#     col("parsed_payload.*")  # unpack all fields from payload
# )

# df_spark_large.show(5)


+--------------------+------+--------+--------+-----+-----+---------+------+---------+--------+-------+----------------+-----------+---------+----------+---+---+--------+-------+--------+-----+-----+---+--------+---+---+----+----+--------+----+-----+--------+
|           timestamp|   hex|alt_baro|alt_geom|   gs|track|baro_rate|squawk|emergency|category|nav_qnh|nav_altitude_mcp|nav_heading|      lat|       lon|nic| rc|seen_pos|version|nic_baro|nac_p|nac_v|sil|sil_type|gva|sda|mlat|tisb|messages|seen| rssi|  flight|
+--------------------+------+--------+--------+-----+-----+---------+------+---------+--------+-------+----------------+-----------+---------+----------+---+---+--------+-------+--------+-----+-----+---+--------+---+---+----+----+--------+----+-----+--------+
|2024-11-05 00:00:...|407799| 39000.0| 40275.0|537.8| 66.4|      0.0|  3405|     none|      A5| 1013.6|         39008.0|       73.1|44.148699| -74.63974|  8|186|     0.1|      2|       1|   10|    2|  3| perhour|  2|  2|

In [5]:
# Load full payload fields into DuckDB from small telemetry dataset
con.execute(f"""
CREATE OR REPLACE TABLE duck_small_telemetry AS
SELECT * 
FROM read_json_auto('{small_telemetry_path}', union_by_name=true, sample_size=-1);
""")

# Preview table
con.execute("SELECT * FROM duck_small_telemetry LIMIT 5").fetchdf()


,type,dt,payload
0,new_adsb,2025-01-31 00:00:00.330843,"{'hex': 'ab35d3', 'flight': 'DAL136 ', 'alt_b..."
1,new_adsb,2025-01-31 00:00:00.336607,"{'hex': 'c03f37', 'flight': 'WJA598 ', 'alt_b..."
2,new_adsb,2025-01-31 00:00:00.336872,"{'hex': 'c06a75', 'flight': 'TSC359 ', 'alt_b..."
3,new_adsb,2025-01-31 00:00:00.337039,"{'hex': 'c078ba', 'flight': 'TSC939 ', 'alt_b..."
4,new_adsb,2025-01-31 00:00:00.337196,"{'hex': 'c027da', 'flight': 'CJT990 ', 'alt_b..."


In [6]:
import time

start = time.time()

# Flatten + group all in one go (inside subquery)
result_df = con.sql("""
SELECT hex, COUNT(*) AS flights
FROM (
    SELECT 
        payload ->> 'hex' AS hex
    FROM duck_small_telemetry
    WHERE payload ->> 'hex' IS NOT NULL
    LIMIT 10000
)
GROUP BY hex
ORDER BY flights DESC
LIMIT 10
""").df()

end = time.time()

# Show result and timing
print(result_df)
print(f"⏱ DuckDB query time: {end - start:.4f} seconds")
print("✅ DuckDB query executed successfully")


      hex  flights
0  89901f      855
1  c04463      615
2  c00a0b      585
3  ab35d3      551
4  c02fe7      546
5  407f2c      505
6  ab7fe6      497
7  a7615f      495
8  c0202d      491
9  a89bec      489
⏱ DuckDB query time: 0.2910 seconds
✅ DuckDB query executed successfully


In [7]:
import time
from pyspark.sql.functions import col

start = time.time()

spark_result_small = df_spark_small_flat.limit(10000) \
    .filter(col("hex").isNotNull()) \
    .groupBy("hex") \
    .count() \
    .orderBy(col("count").desc()) \
    .limit(10) \
    .toPandas()

end = time.time()

print(spark_result_small)
print(f"⚡ Spark (small) query time: {end - start:.4f} seconds")
print("✅ Spark small dataset query completed")


      hex  count
0  89901f    855
1  c04463    615
2  c00a0b    585
3  ab35d3    551
4  c02fe7    546
5  407f2c    505
6  ab7fe6    497
7  a7615f    495
8  c0202d    491
9  a89bec    489
⚡ Spark (small) query time: 1.7658 seconds
✅ Spark small dataset query completed


In [8]:
import pandas as pd
import time


# Load as line-delimited JSON
df_pandas = pd.read_json(small_telemetry_path, lines=True)

# Flatten the nested payload column
pandas_df = pd.json_normalize(df_pandas['payload'])
pandas_df = df_pandas[['dt']].join(pandas_df)
pandas_df.rename(columns={'dt': 'timestamp'}, inplace=True)

# ✅ Take a 10,000-row sample for the benchmark
df_pandas_sample = pandas_df.head(10000)

# 🕒 Run benchmark
start = time.time()

pandas_result = df_pandas_sample[df_pandas_sample["hex"].notnull()] \
    .groupby("hex")["flight"] \
    .count() \
    .reset_index(name="flights") \
    .sort_values("flights", ascending=False) \
    .head(10)

end = time.time()

# ✅ Print result and timing
print(pandas_result)
print(f"🐼 Pandas (small) query time: {end - start:.4f} seconds")
print("✅ Pandas small dataset query completed")


       hex  flights
8   89901f      837
23  c04463      604
17  c00a0b      561
13  ab35d3      551
21  c02fe7      518
2   407f2c      505
10  a7615f      487
14  ab7fe6      468
19  c0202d      467
11  a89bec      437
🐼 Pandas (small) query time: 0.0240 seconds
✅ Pandas small dataset query completed


In [9]:
start = time.time()

duck_result_q2 = con.sql("""
SELECT hex, COUNT(*) AS flights
FROM (
    SELECT 
        payload ->> 'hex' AS hex,
        TRY_CAST(payload ->> 'alt_baro' AS DOUBLE) AS alt_baro,
        TRY_CAST(payload ->> 'lat' AS DOUBLE) AS lat
    FROM duck_small_telemetry
    LIMIT 10000
)
WHERE alt_baro > 30000 AND lat IS NOT NULL
GROUP BY hex
ORDER BY flights DESC
LIMIT 10
""").df()

end = time.time()

print(duck_result_q2)
print(f"🦆 DuckDB query 2 time: {end - start:.4f} seconds")
print("✅ DuckDB query 2 executed successfully")

      hex  flights
0  89901f      842
1  c00a0b      575
2  ab35d3      551
3  c02fe7      531
4  407f2c      500
5  ab7fe6      491
6  a7615f      485
7  c0202d      478
8  48ae03      418
9  c04adc      365
🦆 DuckDB query 2 time: 0.1820 seconds
✅ DuckDB query 2 executed successfully


In [10]:
start = time.time()

spark_result_q2 = df_spark_small_flat.limit(10000) \
    .filter((col("alt_baro") > 30000) & (col("lat").isNotNull()) & (col("hex").isNotNull())) \
    .groupBy("hex") \
    .count() \
    .orderBy(col("count").desc()) \
    .limit(10) \
    .toPandas()

end = time.time()

print(spark_result_q2)
print(f"⚡ Spark query 2 time: {end - start:.4f} seconds")


      hex  count
0  89901f    842
1  c00a0b    575
2  ab35d3    551
3  c02fe7    531
4  407f2c    500
5  ab7fe6    491
6  a7615f    485
7  c0202d    478
8  48ae03    418
9  c04adc    365
⚡ Spark query 2 time: 1.2075 seconds


In [11]:
start = time.time()

pandas_result_q2 = df_pandas_sample[
    (df_pandas_sample["alt_baro"] > 30000) & 
    (df_pandas_sample["lat"].notnull()) & 
    (df_pandas_sample["hex"].notnull())
].groupby("hex")["flight"].count().reset_index(name="flights") \
 .sort_values("flights", ascending=False) \
 .head(10)

end = time.time()

print(pandas_result_q2)
print(f"🐼 Pandas query 2 time: {end - start:.4f} seconds")


       hex  flights
6   89901f      837
14  c00a0b      560
11  ab35d3      551
17  c02fe7      503
2   407f2c      500
8   a7615f      481
12  ab7fe6      468
15  c0202d      467
3   48ae03      414
19  c04adc      332
🐼 Pandas query 2 time: 0.0334 seconds


In [13]:
start = time.time()

duck_window_result = con.sql("""
SELECT *
FROM (
    SELECT 
        payload ->> 'hex' AS hex,
        TRY_CAST(payload ->> 'alt_baro' AS DOUBLE) AS alt_baro,
        TRY_CAST(payload ->> 'lat' AS DOUBLE) AS lat,
        dt AS timestamp,
        ROW_NUMBER() OVER (PARTITION BY payload ->> 'hex' ORDER BY dt DESC) AS rn
    FROM duck_small_telemetry
    WHERE TRY_CAST(payload ->> 'alt_baro' AS DOUBLE) > 30000
      AND CAST(dt AS DATE) = DATE '2025-01-31'
    LIMIT 10000
)
WHERE rn = 1
""").df()

end = time.time()

print(duck_window_result)
print(f"🦆 DuckDB window query time: {end - start:.4f} seconds")
print("✅ DuckDB window query executed successfully")

Empty DataFrame
Columns: [hex, alt_baro, lat, timestamp, rn]
Index: []
🦆 DuckDB window query time: 1.0099 seconds
✅ DuckDB window query executed successfully


In [14]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, to_date

start = time.time()

window_spec = Window.partitionBy("hex").orderBy(col("timestamp").desc())

spark_window_result = df_spark_small_flat \
    .filter((col("alt_baro") > 30000) & 
            (to_date("timestamp") == "2025-01-31") & 
            (col("hex").isNotNull())) \
    .withColumn("rn", row_number().over(window_spec)) \
    .filter("rn = 1") \
    .select("hex", "alt_baro", "lat", "timestamp") \
    .limit(10000) \
    .toPandas()

end = time.time()

print(spark_window_result)
print(f"⚡ Spark window query time: {end - start:.4f} seconds")


        hex alt_baro        lat                   timestamp
0    0101db    34975        NaN  2025-01-31 13:43:05.129863
1    0201a3    36975  44.483363  2025-01-31 23:03:32.190863
2    040198    34250  45.234650  2025-01-31 11:12:25.265773
3    06a070    31000  45.250031  2025-01-31 20:52:12.732107
4    06a081    31000  44.049217  2025-01-31 10:37:55.798193
..      ...      ...        ...                         ...
399  c0884f    30375  44.725156  2025-01-31 00:50:46.500880
400  c08852    31025  45.811359  2025-01-31 19:02:28.012323
401  c08857    36000        NaN  2025-01-31 19:52:20.879995
402  c08863    37525        NaN  2025-01-31 17:19:28.746109
403  c2b3c3    35000  45.303589  2025-01-31 08:30:26.677906

[404 rows x 4 columns]
⚡ Spark window query time: 4.4191 seconds


In [15]:
start = time.time()

# Convert to datetime if not already
df_pandas_sample["timestamp"] = pd.to_datetime(df_pandas_sample["timestamp"])

pandas_window_result = df_pandas_sample[
    (df_pandas_sample["alt_baro"] > 30000) &
    (df_pandas_sample["timestamp"].dt.date == pd.to_datetime("2025-01-31").date()) &
    (df_pandas_sample["hex"].notnull())
].sort_values(["hex", "timestamp"], ascending=[True, False]) \
 .drop_duplicates(subset="hex", keep="first") \
 .loc[:, ["hex", "alt_baro", "lat", "timestamp"]] \
 .head(10000)

end = time.time()

print(pandas_window_result)
print(f"🐼 Pandas window query time: {end - start:.4f} seconds")


         hex alt_baro        lat                  timestamp
404   0c20a8    30125        NaN 2025-01-31 00:01:27.350301
2774  4005c0    37000        NaN 2025-01-31 00:15:42.388500
9915  407f2c    39000  45.314850 2025-01-31 01:31:15.377248
3068  484416    35000        NaN 2025-01-31 00:19:06.486362
6153  48ae03    59100  46.082898 2025-01-31 00:38:44.428345
6934  4ba94d    35000        NaN 2025-01-31 00:55:48.785736
9998  8406d0    34000  43.537189 2025-01-31 01:44:06.312562
8660  89901f    31000  45.604797 2025-01-31 01:13:46.271610
1345  a55ddf    45000  43.659760 2025-01-31 00:05:35.076742
4953  a7615f    39025  45.373260 2025-01-31 00:28:46.781585
4358  a89bec    36000  43.507815 2025-01-31 00:25:51.032281
4939  aa6b9f    37000  45.804749 2025-01-31 00:28:38.594846
2214  ab35d3    37000  45.354355 2025-01-31 00:11:06.475285
4875  ab7fe6    37000  44.497329 2025-01-31 00:27:54.672400
9999  c0054d    35000  44.560730 2025-01-31 01:44:06.312725
6346  c00a0b    37000        NaN 2025-01

C:\Users\raksh\AppData\Local\Temp\ipykernel_5032\810727891.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pandas_sample["timestamp"] = pd.to_datetime(df_pandas_sample["timestamp"])


In [16]:
start = time.time()

duck_date_range_result = con.sql("""
SELECT 
    payload ->> 'hex' AS hex,
    TRY_CAST(payload ->> 'alt_baro' AS DOUBLE) AS alt_baro,
    dt AS timestamp
FROM duck_small_telemetry
WHERE CAST(dt AS DATE) BETWEEN DATE '2025-01-15' AND DATE '2025-01-31'
LIMIT 10000
""").df()

end = time.time()

print(duck_date_range_result.head())
print(f"🦆 DuckDB date range filter time: {end - start:.4f} seconds")
print("✅ DuckDB date range filter executed successfully")

      hex  alt_baro                  timestamp
0  ab35d3   37000.0 2025-01-31 00:00:00.330843
1  c03f37   23925.0 2025-01-31 00:00:00.336607
2  c06a75   24725.0 2025-01-31 00:00:00.336872
3  c078ba   15175.0 2025-01-31 00:00:00.337039
4  c027da   30000.0 2025-01-31 00:00:00.337196
🦆 DuckDB date range filter time: 0.1202 seconds
✅ DuckDB date range filter executed successfully


In [17]:
from pyspark.sql.functions import to_date

start = time.time()

spark_date_range_result = df_spark_small_flat \
    .filter(
        (to_date("timestamp") >= "2025-01-15") & 
        (to_date("timestamp") <= "2025-01-31")
    ) \
    .select("hex", "alt_baro", "timestamp") \
    .limit(10000) \
    .toPandas()

end = time.time()

print(spark_date_range_result.head())
print(f"⚡ Spark date range filter time: {end - start:.4f} seconds")


      hex alt_baro                   timestamp
0  ab35d3    37000  2025-01-31 00:00:00.330843
1  c03f37    23925  2025-01-31 00:00:00.336607
2  c06a75    24725  2025-01-31 00:00:00.336872
3  c078ba    15175  2025-01-31 00:00:00.337039
4  c027da    30000  2025-01-31 00:00:00.337196
⚡ Spark date range filter time: 0.3030 seconds


In [18]:
# Ensure timestamp is in datetime format
df_pandas_sample["timestamp"] = pd.to_datetime(df_pandas_sample["timestamp"])

start = time.time()

pandas_date_range_result = df_pandas_sample[
    (df_pandas_sample["timestamp"].dt.date >= pd.to_datetime("2025-01-15").date()) &
    (df_pandas_sample["timestamp"].dt.date <= pd.to_datetime("2025-01-31").date())
][["hex", "alt_baro", "timestamp"]].head(10000)

end = time.time()

print(pandas_date_range_result.head())
print(f"🐼 Pandas date range filter time: {end - start:.4f} seconds")


      hex alt_baro                  timestamp
0  ab35d3    37000 2025-01-31 00:00:00.330843
1  c03f37    23925 2025-01-31 00:00:00.336607
2  c06a75    24725 2025-01-31 00:00:00.336872
3  c078ba    15175 2025-01-31 00:00:00.337039
4  c027da    30000 2025-01-31 00:00:00.337196
🐼 Pandas date range filter time: 0.0126 seconds


C:\Users\raksh\AppData\Local\Temp\ipykernel_5032\2342127336.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pandas_sample["timestamp"] = pd.to_datetime(df_pandas_sample["timestamp"])


In [19]:
start = time.time()

duck_count_by_date = con.sql("""
SELECT 
    CAST(dt AS DATE) AS date,
    COUNT(*) AS total_records
FROM duck_small_telemetry
GROUP BY date
ORDER BY date
""").df()

end = time.time()

print(duck_count_by_date)
print(f"🦆 DuckDB date-wise count time: {end - start:.4f} seconds")


        date  total_records
0 2025-01-31         301388
1 2025-02-01         305422
2 2025-02-02         358110
3 2025-02-03         263173
4 2025-02-04         282303
5 2025-02-05         319251
🦆 DuckDB date-wise count time: 0.0261 seconds


In [20]:
from pyspark.sql.functions import to_date

start = time.time()

spark_count_by_date = df_spark_small_flat \
    .withColumn("date", to_date("timestamp")) \
    .groupBy("date") \
    .count() \
    .orderBy("date") \
    .toPandas()

end = time.time()

print(spark_count_by_date)
print(f"⚡ Spark date-wise count time: {end - start:.4f} seconds")


         date   count
0  2025-01-31  301388
1  2025-02-01  305422
2  2025-02-02  358110
3  2025-02-03  263173
4  2025-02-04  282303
5  2025-02-05  319251
⚡ Spark date-wise count time: 1.2455 seconds


In [21]:
# Make sure timestamp is datetime
df_pandas_sample["timestamp"] = pd.to_datetime(df_pandas_sample["timestamp"])

start = time.time()

pandas_count_by_date = df_pandas_sample.copy()
pandas_count_by_date["date"] = pandas_count_by_date["timestamp"].dt.date

pandas_count_by_date = pandas_count_by_date \
    .groupby("date")["hex"] \
    .count() \
    .reset_index(name="total_records") \
    .sort_values("date")

end = time.time()

print(pandas_count_by_date)
print(f"🐼 Pandas date-wise count time: {end - start:.4f} seconds")


         date  total_records
0  2025-01-31          10000
🐼 Pandas date-wise count time: 0.0125 seconds


C:\Users\raksh\AppData\Local\Temp\ipykernel_5032\2517065415.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pandas_sample["timestamp"] = pd.to_datetime(df_pandas_sample["timestamp"])


In [24]:
#pattern match

start = time.time()
duck_pattern = con.sql("""
SELECT hex, flight
FROM (
    SELECT payload ->> 'hex' AS hex, payload ->> 'flight' AS flight
    FROM duck_small_telemetry
)
WHERE flight LIKE 'A%'
""").df()
end = time.time()
print(duck_pattern)
print(f"🦆 DuckDB pattern match time: {end - start:.4f} sec")


           hex    flight
0       c02fe7  ACA620  
1       c02fe7  ACA620  
2       c02fe7  ACA620  
3       c02fe7  ACA620  
4       c02fe7  ACA620  
...        ...       ...
397149  440b9c  ABB278  
397150  440b9c  ABB278  
397151  440b9c  ABB278  
397152  440b9c  ABB278  
397153  440b9c  ABB278  

[397154 rows x 2 columns]
🦆 DuckDB pattern match time: 1.9027 sec


In [25]:
start = time.time()
spark_pattern = df_spark_small_flat.filter(col("flight").startswith("A")).select("hex", "flight").toPandas()
end = time.time()
print(f"⚡ Spark pattern match time: {end - start:.4f} sec")


⚡ Spark pattern match time: 5.4441 sec


In [26]:
#nested sub query 

start = time.time()
duck_nested = con.sql("""
SELECT hex, alt_baro FROM (
    SELECT payload ->> 'hex' AS hex, TRY_CAST(payload ->> 'alt_baro' AS DOUBLE) AS alt_baro
    FROM duck_small_telemetry
)
WHERE alt_baro > (
    SELECT AVG(TRY_CAST(payload ->> 'alt_baro' AS DOUBLE)) FROM duck_small_telemetry
)
""").df()
end = time.time()
print(f"🦆 DuckDB nested subquery time: {end - start:.4f} sec")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

🦆 DuckDB nested subquery time: 3.4702 sec


In [ ]:
start = time.time()
avg_alt = df_spark_small_flat.selectExpr("avg(alt_baro)").first()[0]
spark_nested = df_spark_small_flat.filter(col("alt_baro") > avg_alt).select("hex", "alt_baro").toPandas()
end = time.time()
print(f"⚡ Spark nested subquery time: {end - start:.4f} sec")
